# Языковые модели

Какое слово в последовательности вероятнее: 

Поезд прибыл на
* вокзал
* север

Какая последовательность вероятнее:
* Вокзал прибыл поезд на
* Поезд прибыл на вокзал

Языковая модель [language model, LM]  позволяет оценить вероятность следующего слова в последовательности  $P(w_n | w_1, \ldots, w_{n-1})$ и оценить вероятность всей последовательности слов $P(w_1, \ldots, w_n)$.

### Приложения:
* Задачи, в которых нужно обработать сложный и зашумленный вход: распознавание речи, распознавание сканированных и рукописных текстов;
* Исправление опечаток
* Машинный перевод
* Подсказка при наборе 

### Виды моделей:
* Счетные модели
    * цепи Маркова
* Нейронные модели, обычно реккурентные нейронные сети с LSTM/GRU
* seq2seq архитектуры

## Модель $n$-грам

Пусть $w_{1:n}=w_1,\ldots,w_m$ – последовательность слов.

Цепное правило:
$ P(w_{1:m}) = P(w_1) P(w_2 | w_1) P(w_3 | w_{1:2}) \ldots P(w_m | w_{1:m-1}) = \prod_{k=1}^{m} P(w_k | w_{1:k-1}) $

Но оценить $P(w_k | w_{1:k-1})$ не легче! 

Переходим к $n$-грамам: $P(w_{i+1} | w_{1:i}) \approx P(w_{i+1} | w_{i-n:i})  $ , то есть, учитываем $n-1$ предыдущее слово. 

Модель
* униграм: $P(w_k)$
* биграм: $P(w_k | w_{k-1})$
* триграм: $P(w_k | w_{k-1} w_{k-2})$


Т.е. используем Марковские допущения о длине запоминаемой цепочки.

* Вероятность следующего слова в последовательности: $ P(w_{i+1} | w_{1:i}) \approx P(w_{i-n:i}) $
* Вероятность всей последовательности слов: $P(w_{1:n}) = \prod_{k=1}^l P(w_k | w_{k-n+1: k-1}) $

### Качество модели  $n$-грам

Перплексия: насколько хорошо модель предсказывает выборку. Чем ниже значение перплексии, тем лучше.

$PP(\texttt{LM}) = 2 ^ {-\frac{1}{m} \log_2 \texttt{LM} (w_i | w_{1:i-1})}$

## Счетные языковые модели

### ММП оценки вероятностей
$ P_{MLE}(w_k | w_{k-n+1:k-1}) = \frac{\texttt{count}(w_{k-n+1:k-1} w_k )}{\texttt{count}(w_{k-n+1:k-1} )} $

В модели биграм:

$ P_{MLE}(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k )}{\texttt{count}(w_{k-1} )} $

Возникает проблема нулевых вероятностей!

### Аддитивное сглаживание Лапласа

$ P(w_k | w_{k-1}) = \frac{\texttt{count}(w_{k-1} w_k ) + \alpha}{\texttt{count}(w_{k-1} ) + \alpha |V|} $

![AiB](img/aib.png)  

BOS А и Б сидели на трубе EOS

BOS А упало Б пропало EOS

BOS что осталось на трубе EOS




$P($ и $| $ A $) = \frac{1}{2}$

$P($ Б $| $ и $) = \frac{1}{1}$

$P($ трубе $| $ на $) = \frac{2}{2}$

$P($ сидели $| $ Б $) = \frac{1}{2}$

$P($ на $| $ сидели $) = \frac{1}{2}$


## Модели биграм в NLTK

In [1]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
names = [name.strip().lower() for name in open('data/dinos.txt').readlines()]
print(names[:10])

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus', 'abrosaurus', 'abydosaurus', 'acanthopholis', 'achelousaurus', 'acheroraptor']


In [4]:
chars = [char  for name in names for char in name]
freq = nltk.FreqDist(chars)

print(list(freq.keys()))

['a', 'c', 'h', 'e', 'n', 'o', 's', 'u', 'r', 'd', 'y', 'x', 'b', 'l', 'i', 't', 'p', 'v', 'm', 'g', 'f', 'j', 'k', 'w', 'z', 'q']


In [5]:
cfreq = nltk.ConditionalFreqDist(nltk.bigrams(chars))
cfreq['a']

FreqDist({'a': 26,
          'b': 32,
          'c': 109,
          'd': 41,
          'e': 48,
          'f': 7,
          'g': 44,
          'h': 21,
          'i': 26,
          'j': 8,
          'k': 22,
          'l': 146,
          'm': 74,
          'n': 354,
          'o': 27,
          'p': 96,
          'q': 3,
          'r': 131,
          's': 187,
          't': 213,
          'u': 792,
          'v': 34,
          'w': 10,
          'x': 12,
          'y': 14,
          'z': 10})

In [6]:
cprob = nltk.ConditionalProbDist(cfreq, nltk.MLEProbDist)
print('p(a a) = %1.4f' %cprob['a'].prob('a'))
print('p(a b) = %1.4f' %cprob['a'].prob('b'))
print('p(a u) = %1.4f' %cprob['a'].prob('u'))

p(a a) = 0.0105
p(a b) = 0.0129
p(a u) = 0.3185


In [7]:
from math import log
log(cprob['a'].prob('a')) + log(cprob['a'].prob('b')) + log(cprob['a'].prob('c'))

-12.041317008359863

In [8]:
l = sum([freq[char] for char in freq])
def unigram_prob(char):
    return freq[char] / l
print('p(a) = %1.4f' %unigram_prob('a'))

p(a) = 0.1354


In [9]:
[bi for bi in nltk.bigrams('aachenosaurus')]

[('a', 'a'),
 ('a', 'c'),
 ('c', 'h'),
 ('h', 'e'),
 ('e', 'n'),
 ('n', 'o'),
 ('o', 's'),
 ('s', 'a'),
 ('a', 'u'),
 ('u', 'r'),
 ('r', 'u'),
 ('u', 's')]

#### Задание 1

1. Напишите функцию для оценки вероятности имени динозавра. 
2. Найдите наиболее вероятное имя динозавра из данного списка. 

In [10]:
cprob["a"].generate()

'u'

#### Задание 2

Напишите функцию для генерации нового имени динозавра фиксированной длины.

## Нейронные языковые модели

* Вход: $n$-грамы $w_{1:k}$
* $v(w_i)$ – эмбеддинг слова $w_i$, $v(w_i) \in \mathbb{R}^{d_{emb}}$, $d_{emb}$ – размерность эмбеддинга, $v(w) = E_{[w]}$
* $x = [v(w_1), v(w_2), \ldots , v(w_k)]$

$\widehat{y} = P(w_i | w_{1:k} ) = \texttt{LM}(w_{1:k}) = \texttt{softmax}(hW^2 +b^2)$

$h = g(xW^1+b^1)$

$w_i \in V$, $E \in \mathbb{R}^{|V|\times d_{emb}}, W^1 \in \mathbb{R}^{k \cdot d_{emb} \times d_{hid}}, b^1 \in \mathbb{R} ^ {d_{hid}}, W^2 \in \mathbb{R}^{d_{hid} \times |V|}, b^2 \in \mathbb{R} ^ {|V|}$

![nnlm](img/nlm1.png)

### Семплирование в нейронных языковых моделях 
### (Генерация текстов с помощью нейронных языковых моделей)

1. Задать начальную последовательность символов длины $k$ (/слов)
2. Предсказать распределение вероятностей слов с условием на $k$ предыдущих слов
3. 1. Выбрать слово с наибольшей вероятностью
3. 2. Выбрать слово по предсказаному распределению
4. Сдвинуть окно на одно слово и повторить 

#### Линейный поиск  (beam search)
Всегда помним $h$ наиболее вероятных гипотез:
1. Для генерации первого слова в последоватительности генерируем $h$ кандидатов, а не 1
2. Генерируем $h \times h$ кандидатов для второго слова и храним только $h$ наиболее вероятных


![nnlm](img/nlm2.png)

In [11]:
alphabet = list(set(chars))
print('total chars:', len(alphabet))

total chars: 26


In [12]:
maxlen = 5
step = 1
ngrams = []
next_chars = []
for name in names:
    for i in range(0, len(name) - maxlen, step):
        ngrams.append(' '.join([char for char in name[i: i + maxlen]]))
        next_chars.append(name[i + maxlen])
print('nb ngrams:', len(ngrams))
print(ngrams[0],next_chars[0])
print(ngrams[1],next_chars[1])

nb ngrams: 10701
a a c h e n
a c h e n o


In [13]:
tokenizer = Tokenizer(num_words=len(alphabet))
tokenizer.fit_on_texts(ngrams)

sequences = tokenizer.texts_to_sequences(ngrams)
X_train = pad_sequences(sequences, maxlen=maxlen)
sequences = tokenizer.texts_to_sequences(next_chars)
y_train = tokenizer.sequences_to_matrix(sequences)
X_train[0]

array([ 1,  1, 12, 11,  7], dtype=int32)

In [14]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [16]:
model = Sequential()

model.add(Embedding(len(alphabet), 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(64, activation = 'softmax'))
model.add(Dropout(0.2))
model.add(Dense(32, activation = 'softmax'))
model.add(Dropout(0.2))
model.add(Dense(len(alphabet), activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train,y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 0)

In [18]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)), p = preds)
    return probas

In [19]:
import random
import sys
import numpy as np

generated = ''
seed = 'katya'
generated += seed
print('----- Generating with seed: "' + seed + '"')
print(generated)

for i in range(8):
    sequences = tokenizer.texts_to_sequences([' '.join([char for char in generated[-maxlen:]])])
    X_pred = pad_sequences(sequences, maxlen=maxlen)
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = index_char[next_index]
    generated += next_char
    print(generated)

----- Generating with seed: "katya"
katya
katyao
katyaoj
katyaojm
katyaojmz
katyaojmzr
katyaojmzrs
katyaojmzrst
katyaojmzrstk


#### Задание 3

Измените код выше так, чтобы генерировались панграмы – имена динозавров, не содержащие повторяющихся букв

#### Задание 4

Измените функцию семлирования `sample`: добавьте параметр `t`, изпольузуемый для шкалирования вероятностей  `preds`: ```
preds /= t
``` 

Как использование этого параметра влияет на генерируемые имена?

### Рекуррентные нейронные языковые модели

RNN позволяют уйти от Марковских допущений и позволяют учитывать предысторию произвольной длины.

$x_{1:n} = x_1, x_2, \ldots, x_n$, $x_i \in \mathbb{R}^{d_{in}}$

$y_n = RNN(x_{1:n})$, $y_n \in \mathbb{R}^{d_{out}}$

Для каждого префикса $x_{i:i}$ $y_i$ – выходной вектор.

$y_i = RNN(x_{1:i})$

$y_{1:n} = RNN^{*}(x_{1:n})$, $y_i \in \mathbb{R}^{d_{out}}$

![rnn](img/rnn1.png)

$R$ –  рекурсивная функция с двумя входами: $x_i$ и $s_{i-1}$ (вектор состояния)

$RNN^{*}(x_{1:n}, s_0) = y_{1:n}$

$y_i = O(s_i)$

$s_i = R(s_{i-1}, x_i)$

$s_i = R(s_{i-1}, x_i) = g(s_{i-1}* W^s + x_i W^x +b$

$x_i \in \mathbb{R}^{d_{in}}$, $y_i \in \mathbb{R}^{d_{out}}$, $s_i \in \mathbb{R}^{d_{out}}$

$W^x \in \mathbb{R}^{d_{in} \times d_{in}}$, $W^s \in \mathbb{R}^{d_{out} \times d_{out}}$

#### Обучение RNN: backpropogation through time (BPTT)

![rnn](img/rnn3.png)

#### Сценарии использования RNN

* Acceptor: только $y_n$, используемый для дальнейшего предсказания / классификации

![rnn](img/rnn5.png)


* Encoder: только $y_n$

![rnn](img/rnn5.png)

* Transducer: выход $t_i$ для каждого входа $x_{1:i}$ – языковые модели,  sequence labelling

![rnn](img/rnn2.png)

#### Двунаправленная рекуррентная нейронная сеть

$x_{1:n}$ – входная последовательность

$s_i^f$ –  "прошлое / прямое состояние" – основано на $x_{1:i}$

$s_i^b$ –  "будущее / обратное состояние" – основано на $x_{n:i}$

$y_i = [O(s_i^f), O(s_i^b)] = [y_i^f, y_i^b]$

![rnn](img/birnn.png)

#### Управляемые архитектуры

RNN трудно обучать: проблема исчезающего градиента. Уйти от нее помогают управляемые нейроны специального вида: LSTM и GRU.

$s_i$ – память нейронной сети. Каждое использование $R$ считывает и видоизменяет всю память. 

Управляемый доступ к памяти: $g \in {0,1}^n$:

$s_{i+1} \leftarrow g \odot x + (1-g) \odot s_{i}$

Дифференцируемое управление:

$g \in \mathbb{R}^n $:

$s_{i+1} \leftarrow \sigma(g) \odot x + (1-g) \odot s_{i}$

http://colah.github.io/posts/2015-08-Understanding-LSTMs/
    

In [132]:
from keras.utils import to_categorical
import numpy as np

X_names = ['< ' + ' '.join(name) for name in names]
Y_names = [' '.join(name) + ' >' for name in names]
maxlen = max([len(name) for name in names])+1

In [133]:
X_names[1]

'< a a r d o n y x'

In [139]:
tokenizer = Tokenizer(num_words=len(alphabet) + 2, filters = '')
tokenizer.fit_on_texts(X_names+Y_names)

sequences = tokenizer.texts_to_sequences(X_names)
X_train = pad_sequences(sequences, maxlen=maxlen, padding='post', value = tokenizer.word_index['>'])


sequences = tokenizer.texts_to_sequences(Y_names)
Y_train = pad_sequences(sequences, padding='post')


Y_train_cat  = [to_categorical(sent, num_classes=len(alphabet)+2) for sent in Y_train]
Y_train =  np.asarray(Y_train_cat)

In [140]:
print(X_names[0])
print(Y_names[0])

print(X_train[0])


print(X_train.shape)
print(Y_train.shape)

print(tokenizer.word_index['<'])
print(tokenizer.word_index['>'])

< a a c h e n o s a u r u s
a a c h e n o s a u r u s >
[10  1  1 15 14  8  6  4  2  1  3  5  3  2 11 11 11 11 11 11 11 11 11 11
 11 11 11]
(1536, 27)
(1536, 27, 28)
10
11


In [141]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [142]:
model = Sequential()

model.add(Embedding(len(alphabet)+3, 100, input_length=maxlen, trainable = True))
model.add(LSTM(128, return_sequences = True))
model.add(TimeDistributed(Dense(len(alphabet)+2, activation = 'softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [143]:
for iteration in range(1, 100):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, Y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 1)

Epoch 1/1
1536/1536 [==============================] - 2s 2ms/step - loss: 3.3296 - acc: 0.0228
Epoch 1/1
1536/1536 [==============================] - 1s 590us/step - loss: 3.3007 - acc: 0.5434
Epoch 1/1
1536/1536 [==============================] - 1s 767us/step - loss: 3.2708 - acc: 0.5279
Epoch 1/1
1536/1536 [==============================] - 1s 713us/step - loss: 3.2382 - acc: 0.5205
Epoch 1/1
1536/1536 [==============================] - 1s 716us/step - loss: 3.2008 - acc: 0.5205
Epoch 1/1
1536/1536 [==============================] - 1s 606us/step - loss: 3.1559 - acc: 0.5205
Epoch 1/1
1536/1536 [==============================] - 1s 599us/step - loss: 3.0997 - acc: 0.5205
Epoch 1/1
1536/1536 [==============================] - 1s 605us/step - loss: 3.0268 - acc: 0.5205
Epoch 1/1
1536/1536 [==============================] - 1s 597us/step - loss: 2.9290 - acc: 0.5205
Epoch 1/1
1536/1536 [==============================] - 1s 616us/step - loss: 2.7941 - acc: 0.5205
Epoch 1/1
1536/1536 [=

Epoch 1/1
1536/1536 [==============================] - 1s 796us/step - loss: 1.4059 - acc: 0.5897
Epoch 1/1
1536/1536 [==============================] - 1s 892us/step - loss: 1.4026 - acc: 0.5896
Epoch 1/1
1536/1536 [==============================] - 1s 792us/step - loss: 1.3994 - acc: 0.5883
Epoch 1/1
1536/1536 [==============================] - 3s 2ms/step - loss: 1.3966 - acc: 0.5875
Epoch 1/1
1536/1536 [==============================] - 2s 1ms/step - loss: 1.3938 - acc: 0.5878
Epoch 1/1
1536/1536 [==============================] - 1s 870us/step - loss: 1.3908 - acc: 0.5892
Epoch 1/1
1536/1536 [==============================] - 1s 782us/step - loss: 1.3876 - acc: 0.5916
Epoch 1/1
1536/1536 [==============================] - 1s 829us/step - loss: 1.3847 - acc: 0.5948
Epoch 1/1
1536/1536 [==============================] - 1s 701us/step - loss: 1.3819 - acc: 0.5974
Epoch 1/1
1536/1536 [==============================] - 1s 748us/step - loss: 1.3791 - acc: 0.5987
Epoch 1/1
1536/1536 [===

In [144]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)+2), p = preds)
    return probas

In [149]:
generated = ''
seed = '<'
generated += seed + ' '
print('----- Generating with seed: "' + seed + '"')
print(generated)


for i in range(15): 
    sequences = tokenizer.texts_to_sequences([seed])
    X_pred = pad_sequences(sequences, maxlen=maxlen, padding = 'post')

    preds = model.predict(X_pred, verbose=0)[0]
    samples = [sample(p) for p in preds]
    next_index = samples[i]
    next_char = index_char[next_index+1]
    generated += next_char + ' '
    print(generated)
    seed += next_char
    if next_char == '>':
        break
    

----- Generating with seed: "<"
< 
< x 
< x a 
< x a f 
< x a f k 
< x a f k o 
< x a f k o i 
< x a f k o i s 
< x a f k o i s h 
< x a f k o i s h a 
< x a f k o i s h a e 
< x a f k o i s h a e m 
< x a f k o i s h a e m l 
< x a f k o i s h a e m l o 
< x a f k o i s h a e m l o r 
< x a f k o i s h a e m l o r u 


In [118]:
index_char

{1: 'a',
 2: 's',
 3: 'u',
 4: 'o',
 5: 'r',
 6: 'n',
 7: 'i',
 8: 'e',
 9: 't',
 10: '<',
 11: '>',
 12: 'l',
 13: 'p',
 14: 'h',
 15: 'c',
 16: 'g',
 17: 'd',
 18: 'm',
 19: 'y',
 20: 'b',
 21: 'k',
 22: 'v',
 23: 'x',
 24: 'z',
 25: 'j',
 26: 'w',
 27: 'f',
 28: 'q'}

#### Задание 5

Измените функцию семлирования `sample`: сейчас в ней используется простой выбор наиболее вероятного следующего элемента. Замените его на лучевой поиск [beam search].

Как использование этой функции семплирования влияет на генерируемые имена?

In [ ]:
# решение 1.1
def name_prob(name):
    p = unigram_prob(name[0])
    for i in range(len(name)-1):
        p *=  cprob[name[i]].prob(name[i+1])
    return p

name_prob(names[0])

In [ ]:
# решение 2
def generate_name(cprob, first_char, num_chars):
    name = ''
    name += first_char
    for i in range(num_chars):
        char = cprob[first_char].generate()
        name += char
        first_char = char
    return name

generate_name(cprob, 't', 9)

In [ ]:
# решение 3
for i in range(5):
    sequences = tokenizer.texts_to_sequences([' '.join([char for char in generated[-maxlen:]])])
    X_pred = pad_sequences(sequences, maxlen=maxlen)
    preds = model.predict(X_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = index_char[next_index]
    if next_char not in generated:
        generated += next_char
        print(generated)
    else:
        preds[next_index] = 10e-5
        next_index = sample(preds)
        next_char = index_char[next_index]
        generated += next_char
        print(generated)

In [ ]:
# решение 4
def sample(preds, t=1.2):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / t
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)), p = preds)
    return probas